## Import

In [1]:
import numpy as np
import pandas as pd
import pickle
import timeit
from sklearn import tree
from sklearn import metrics
from sklearn.model_selection import train_test_split
# import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pylab as plt
import os
from collections import Counter

## Parameters

In [2]:
####################################################################################################################
#################################################Parameters#########################################################
####################################################################################################################
location = 'Datasets/new2_toy_pca4' #adult #new_adult_pca1
file = 'toy_pca'  #toy_pca
exp = 'new5_toy_pca4'
dataset = "toy_pca" #toy_pca 
# file='new_adult_pca3' 


train_data= '_Train_Data.csv'
test_data= '_Test_Data.csv'
val_data ='_Val_Data.csv'
# learner_model = 'XGB'
# lstm_experiment_dict = {0: [0.0,5000], 1: [0.5,25000]}
lstm_experiment_dict = {0: [0.0,7500], 1: [0.5,37500] ,2:[0.67,50000]}
# , 2: [0.67,33500]}

# lstm_experiment_dict = {0: [0.0,5000],
#                     1: [0.1,5000],
#                     2: [0.5,25000],
#                     3: [0.67,33500],
#                     4: [0.75,37500],
#                     5: [0.8,40000],
#                     6: [0.85,42500],
#                     8: [0.9,45000]}

lstm_pickle_name = "experiment_dict.pickle"
sffs_pickle_name = "_sffs_mlxtend_runtime4.pickle"



## Merage experiment dictionaries 

In [3]:
# 0.67_50000

In [4]:
# file = 'pca4'
# # exp = '0.0_7500'
# # exp = '0.5_37500'
# exp = '0.67_50000'
# plus1 = 15 
# # plus2 = 15
# # # plus3 = 27
# # # # # import pickle5 as pickle
# with open(f'Experiments/2new2_toy_{file}_toy_pca_lstm_{exp}'+  '/' + lstm_pickle_name, "rb") as handle:
#     experiment_dict = pickle.load(handle)
# with open(f'Experiments/2new2_toy_{file}_toy_pca_lstm_{exp}'+  '/' + lstm_pickle_name, "rb") as handle:
#     experiment_dict1 = pickle.load(handle)
# # with open(f'Experiments/3new2_toy_{file}_toy_pca_lstm_{exp}'+  '/' + lstm_pickle_name, "rb") as handle:
# #     experiment_dict2 = pickle.load(handle)
# # # # with open(f'Experiments/4new3_toy_{file}_toy_pca_lstm_{exp}'+  '/' + lstm_pickle_name, "rb") as handle:
# # # #     experiment_dict3 = pickle.load(handle)
       
# experiment_dict.update(dict((key + plus1 , value) for (key, value) in experiment_dict1.items()))
# # experiment_dict.update(dict((key + plus2 , value) for (key, value) in experiment_dict2.items()))
# # # # # experiment_dict.update(dict((key + plus3 , value) for (key, value) in experiment_dict3.items()))

# os.makedirs(f'Experiments_final/new5_toy_{file}_toy_pca_lstm_{exp}', exist_ok=True) 
# with open(f'Experiments_final/new5_toy_{file}_toy_pca_lstm_{exp}'+  '/' + lstm_pickle_name, 'wb') as handle:
#     pickle.dump(experiment_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
# experiment_dict

In [5]:
# with open(location + '/' + str(file) +  sffs_pickle_name, "rb") as handle:
#     experiment_dict = pickle.load(handle)
# experiment_dict

In [6]:
# with open('Experiments/toy_pca2_toy_pca_lstm_0.67_33500'+  '/' + lstm_pickle_name, "rb") as handle:
#     experiment_dict = pickle.load(handle)
# experiment_dict

## Upload experiments results

In [7]:

d = {}
run_time = {}
print(f"lstm")       
for value in lstm_experiment_dict.values(): 
    with open(f'Experiments_final/{exp}_{str(file)}_lstm_{str(value[0])}_{str(value[1])}/' + lstm_pickle_name, "rb") as handle:
        experiment_dict = pickle.load(handle)
    print(f"discount factor {value[0]}")
    d_l = []
    l = [value[0] for value in experiment_dict.values()]
    runtime = [value[1] for value in experiment_dict.values()]

    c = Counter(map(tuple, l))
    c = c.most_common()
    for x in c:
        count = x[1]
        policy = list(x[0])
        if -999 in policy:
            policy.remove(-999)
        d_l.append([count,policy])
    d["lstm_" + str(value[0])] = d_l
    run_time["lstm_" + str(value[0])] = np.mean(runtime)

if dataset == 'adult':
    print(f"DQN")       
    for value in lstm_experiment_dict.values(): 
        with open(f'Experiments_final/{exp}_{str(file)}_DQN_{str(value[0])}_{str(value[1])}/' + lstm_pickle_name, "rb") as handle:
            experiment_dict = pickle.load(handle)
        print(f"discount factor {value[0]}")
        d_l = []
        l = [value[0] for value in experiment_dict.values()]
        runtime = [value[1] for value in experiment_dict.values()]
        c = Counter(map(tuple, l))
        c = c.most_common()
        for x in c:
            count = x[1]
            policy = list(x[0])
            policy.remove(-999)
            d_l.append([count,policy])
        d["DQN_" + str(value[0])] = d_l
        run_time["DQN_" + str(value[0])] = np.mean(runtime)

print(f"sffs")       
with open(location + '/' + str(file) +  sffs_pickle_name, "rb") as f:
    sffs = pickle.load(f) 
d_l = []
l = [list( dict.fromkeys(value[0]) ) for value in sffs.values()]
runtime = [value[1]  for value in sffs.values()]
    
c = Counter(map(tuple, l))
c = c.most_common()
for x in c:
    count = x[1]
    policy = list(x[0])
    d_l.append([count,policy])
d['sffs'] = d_l    
run_time['sffs'] = np.mean(runtime)


df =[]
for key, value in d.items():
    for val in value:
        df.append([key, val[0], val[1]])
df = pd.DataFrame(df, columns =['method', 'policy_count', 'policy'])
df["policy_sort"] = df['policy'].apply(lambda x: sorted(x))


df_rt =[]
for key, value in run_time.items():
    df_rt.append([key, value])
df_rt = pd.DataFrame(df_rt, columns =['method', 'avg_run_time'])


lstm
discount factor 0.0
discount factor 0.5
discount factor 0.67
sffs


In [8]:
df["policy_sort_string"] = df['policy_sort'].astype(str)

In [9]:
df

,method,policy_count,policy,policy_sort,policy_sort_string
0,lstm_0.0,14,"[2, 3, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]"
1,lstm_0.0,8,"[3, 2, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]"
2,lstm_0.0,6,"[1, 3, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]"
3,lstm_0.0,2,"[3, 1, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]"
4,lstm_0.5,12,"[2, 3, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]"
5,lstm_0.5,10,"[3, 2, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]"
6,lstm_0.5,2,"[1, 3, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]"
7,lstm_0.5,2,"[3, 1, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]"
8,lstm_0.67,10,"[2, 3, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]"
9,lstm_0.67,4,"[1, 3, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]"


## Get unique policies

In [10]:
d1 = [value for value in d.values()]
po = []
for val in d1:
    for value in val:
        p = value[1].copy()
        p.sort()
        po.append(p)

In [11]:
policy_sort_unique = []
c = Counter(map(tuple, po))
for x in c:
    policy = list(x)
    policy_sort_unique.append(policy)
    

In [12]:
policy_sort_unique

[[0, 1, 2, 3]]

## Calculate importance and val accuracy

### Load Data

In [13]:
####################################################################################################################
#################################################  DATA PREP ####################################################
####################################################################################################################
if dataset == 'toy_pca':
    train_df = pd.read_csv(location + '/' + file + train_data)
else:
    train_df = pd.read_csv(location + '/' + file + train_data)
    test_df = pd.read_csv(location + '/' + file + test_data)
    val_df = pd.read_csv(location + '/' + file + val_data)

In [14]:
train_df.head(2)

,0,1,2,3,4,5,6,7,8,PC4,label
0,0.248943,-1.478499,-1.437553,-0.547820,0.168693,0.198845,0.093123,-2.444173,0.796576,-0.179926,0
1,0.158677,1.267849,0.448522,0.860439,-0.160290,-0.319902,-2.654472,0.132599,0.036832,0.322281,0


In [15]:
if dataset == 'adult':
    print(val_df.head(2))

In [16]:
if dataset == 'adult':
    print(test_df.head(2))

### importance 

In [17]:
X_train = train_df.iloc[:,:-1]
y_train = train_df.iloc[:,-1]

forest = tree.DecisionTreeClassifier(random_state=0)
forest.fit(X_train, y_train)
importances = forest.feature_importances_
main_indices = np.argsort(importances)[::-1]
final_list = []
for f in range(X_train.shape[1]):
    if importances[main_indices[f]]>=0.1 : #importances.mean():
        print("%d. feature %d (%f)" % (f + 1, main_indices[f], importances[main_indices[f]]))
        final_list.append( main_indices[f])
df.loc[len(df)] = ['importance', 1, final_list, sorted(final_list), sorted(final_list)]
df["policy_sort_string"] = df['policy_sort'].astype(str)
policy_sort_unique.append(sorted(final_list)) if sorted(final_list) not in policy_sort_unique else policy_sort_unique


1. feature 2 (0.261074)
2. feature 3 (0.251678)
3. feature 1 (0.181427)
4. feature 0 (0.166327)


[[0, 1, 2, 3]]

In [18]:
if file == 'toy_pca':
    df.loc[len(df)] = ['real', 1, [1,2,3,0], sorted([1,2,3,0]), sorted([1,2,3,0])]
    df["policy_sort_string"] = df['policy_sort'].astype(str)
    policy_sort_unique.append(sorted([1,2,3,0])) if sorted([1,2,3,0]) not in policy_sort_unique else policy_sort_unique

In [19]:
policy_sort_unique

[[0, 1, 2, 3]]

### Functions 

In [20]:
# Function that separate the episode data into features and label
def data_separate (dataset):
    global X
    global y    
    X = dataset.iloc[:,0:-1]  # all rows, all the features and no labels
    y = dataset.iloc[:, -1]  # all rows, label only
    return X,y

In [21]:
def Learner(X_train, X_test,y_train, y_test, learner_model):
#     global learner
#     global y_pred
    
    if learner_model == 'DT':
        learner = tree.DecisionTreeClassifier(random_state=0)
        learner = learner.fit(X_train, y_train)   #remove batch_index column
        y_pred = learner.predict(X_test)  
    elif learner_model == 'RF':
        learner = RandomForestClassifier()
        learner = learner.fit(X_train, y_train)  
        y_pred = learner.predict(X_test)  
        
    elif learner_model == 'GB':
        learner = GradientBoostingClassifier(random_seed=0)
        learner = learner.fit(X_train, y_train)  
        y_pred = learner.predict(X_test)  
    elif learner_model == 'XGB':
        if file in ['diabetic_data','toy_pca']:
            objective='multi:softprob'       
            num_class= 3
            max_depth=2
        else:
            objective='binary:logistic'
            num_class=1
            max_depth=2
        learner = xgb.XGBRegressor(objective=objective, max_depth=max_depth, n_estimators=350, nthread=8,
                              eval_metric='logloss', eta=0, early_stopping_rounds=100, num_class=num_class)
        learner = learner.fit(X_train, y_train)  
        y_pred = learner.predict(X_test)    
    if  learner_model == 'XGB':
        if file in ['diabetic_data','toy_pca']:
            y_pred = np.asarray([np.argmax(line) for line in y_pred])
        accuracy=metrics.accuracy_score(y_test, (y_pred > 0.5).astype(int))
    else:
        accuracy=metrics.accuracy_score(y_test, y_pred)
    print ("accuracy: {}".format(accuracy))
    print ("numbrt of features: {}".format(X_train.shape[1]))

    return accuracy

In [22]:
# learner = GradientBoostingClassifier(random_state=0)
# policy  = [1, 9, 2, 3, 0]
# print(policy)
# X_train_policy = X_train.iloc[:,policy]
# accuracy = round(cross_val_score(learner, X_train_policy, y_train, cv=10).mean(),3)
# print(accuracy)

In [23]:
# accuracy

### Train - Validation results 

In [24]:
if dataset == 'toy_pca':
    policy_sort_unique_val_results = []
    
    from sklearn.model_selection import cross_val_score

    learner = GradientBoostingClassifier(random_state=0)
    X_train,y_train = data_separate(train_df)

    for policy in policy_sort_unique: 
#     for policy in ([[1, 2, 3, 0],[9, 2, 1, 0, 3, 5],[9, 2, 1, 3, 0],[9, 2, 0, 3]]):
        print(policy)
        X_train_policy = X_train.iloc[:,policy]
#         a = cross_val_score(learner, X_train_policy, y_train, cv=3)
        accuracy = round(cross_val_score(learner, X_train_policy, y_train, cv=3).mean(),3)
        print (accuracy)
        policy_sort_unique_val_results.append([policy,accuracy])

[0, 1, 2, 3]
0.829


In [25]:
# by_Convergence

In [26]:
if dataset == 'adult':
    learner = 'GB' 
    
    data_for_test = val_df.append(test_df)
    policy_sort_unique_val_results = []
    X_train,y_train = data_separate(train_df)
    X_test, y_test = data_separate(data_for_test)
    
    for policy in policy_sort_unique: 
        X_train_policy = X_train.iloc[:,policy]
        X_test_policy  = X_test.iloc[:,policy]
        accuracy = Learner(X_train_policy, X_test_policy, y_train, y_test, learner)
        policy_sort_unique_val_results.append([policy,accuracy])

    policy_sort_unique_val_results

In [27]:
if dataset == 'adult':
    policy_sort_unique_val_results = pd.DataFrame(policy_sort_unique_val_results, columns =['policy_sort_string', 'val_accuracy'])
    policy_sort_unique_val_results["policy_sort_string"] = policy_sort_unique_val_results['policy_sort_string'].astype(str)
    policy_sort_unique_val_results
elif dataset == 'toy_pca':
    policy_sort_unique_val_results = pd.DataFrame(policy_sort_unique_val_results, columns =['policy_sort_string', 'cv_accuracy'])
    policy_sort_unique_val_results["policy_sort_string"] = policy_sort_unique_val_results['policy_sort_string'].astype(str)
    policy_sort_unique_val_results

In [28]:
df  = pd.merge(df, policy_sort_unique_val_results, how="left", on=["policy_sort_string"])

In [29]:
df

,method,policy_count,policy,policy_sort,policy_sort_string,cv_accuracy
0,lstm_0.0,14,"[2, 3, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
1,lstm_0.0,8,"[3, 2, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
2,lstm_0.0,6,"[1, 3, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
3,lstm_0.0,2,"[3, 1, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
4,lstm_0.5,12,"[2, 3, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
5,lstm_0.5,10,"[3, 2, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
6,lstm_0.5,2,"[1, 3, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
7,lstm_0.5,2,"[3, 1, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
8,lstm_0.67,10,"[2, 3, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
9,lstm_0.67,4,"[1, 3, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829


### Train - test results 

In [30]:
# income = test_df['income']
# del test_df['income']
# test_df['Batch_Index'] = 0
# test_df['income'] = income
# test_df

In [31]:
if dataset == 'adult':
    learner = 'GB' 
    policy_sort_unique_test_results = []
    data_for_test = val_df.append(test_df)
    X_train,y_train = data_separate(train_df)
    X_test, y_test = data_separate(data_for_test)
    
#     train_val_df = train_df.copy()

#     X_train,y_train = data_separate(train_val_df)
#     X_test, y_test = data_separate(test_df)
    for policy in policy_sort_unique: 
        X_train_policy = X_train.iloc[:,policy]
        X_test_policy  = X_test.iloc[:,policy]
        accuracy = Learner(X_train_policy, X_test_policy, y_train, y_test, learner)
        policy_sort_unique_test_results.append([policy,accuracy])

    policy_sort_unique_test_results

In [32]:
if dataset == 'adult':
    policy_sort_unique_test_results = pd.DataFrame(policy_sort_unique_test_results, columns =['policy_sort_string', 'test_accuracy'])
    policy_sort_unique_test_results["policy_sort_string"] = policy_sort_unique_test_results['policy_sort_string'].astype(str)
    df  = pd.merge(df, policy_sort_unique_test_results, how="left", on=["policy_sort_string"])

In [33]:
df

,method,policy_count,policy,policy_sort,policy_sort_string,cv_accuracy
0,lstm_0.0,14,"[2, 3, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
1,lstm_0.0,8,"[3, 2, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
2,lstm_0.0,6,"[1, 3, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
3,lstm_0.0,2,"[3, 1, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
4,lstm_0.5,12,"[2, 3, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
5,lstm_0.5,10,"[3, 2, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
6,lstm_0.5,2,"[1, 3, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
7,lstm_0.5,2,"[3, 1, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
8,lstm_0.67,10,"[2, 3, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
9,lstm_0.67,4,"[1, 3, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829


* summary 

In [34]:
if dataset == 'adult':
    by_Convergence = pd.DataFrame([['No','No','Importance',0, 
      df[(df.method=='importance')].policy.values[0], 
      df[(df.method=='importance')].test_accuracy.values[0], '1/1'],
    ['Yes','No','SFFS',0,
    df[(df.method=='sffs')& (df.policy_count==df[df.method=='sffs'].policy_count.max())].policy.values[0],
    df[(df.method=='sffs')& (df.policy_count==df[df.method=='sffs'].policy_count.max())].test_accuracy.values[0],
    f'{df[df.method=="sffs"].policy_count.max()}/30'],
    ['No','Yes','Deep SARSA',0,
    df[(df.method=='DQN_0.0')& (df.policy_count==df[df.method=='DQN_0.0'].policy_count.max())].policy.values[0],
    df[(df.method=='DQN_0.0')& (df.policy_count==df[df.method=='DQN_0.0'].policy_count.max())].test_accuracy.values[0],
    f'{df[df.method=="DQN_0.0"].policy_count.max()}/30'],
    ['No','Yes','Deep SARSA',0.5,
    df[(df.method=='DQN_0.5')& (df.policy_count==df[df.method=='DQN_0.5'].policy_count.max())].policy.values[0],
    df[(df.method=='DQN_0.5')& (df.policy_count==df[df.method=='DQN_0.5'].policy_count.max())].test_accuracy.values[0],
    f'{df[df.method=="DQN_0.5"].policy_count.max()}/30'],
    ['Yes','Yes','Deep LSTM SARSA',0,
    df[(df.method=='lstm_0.0')& (df.policy_count==df[df.method=='lstm_0.0'].policy_count.max())].policy.values[0],
    df[(df.method=='lstm_0.0')& (df.policy_count==df[df.method=='lstm_0.0'].policy_count.max())].test_accuracy.values[0],
    f'{df[df.method=="lstm_0.0"].policy_count.max()}/30'],
    ['Yes','Yes','Deep LSTM SARSA',0.5,
    df[(df.method=='lstm_0.5')& (df.policy_count==df[df.method=='lstm_0.5'].policy_count.max())].policy.values[0],
    df[(df.method=='lstm_0.5')& (df.policy_count==df[df.method=='lstm_0.5'].policy_count.max())].test_accuracy.values[0],
    f'{df[df.method=="lstm_0.5"].policy_count.max()}/30']  ]      
                                  ,
    columns = ['Memory','Future','Method','DF','policy','Test accuracy','Convergence'])
    
elif dataset == 'toy_pca':    
    
    by_Convergence = pd.DataFrame([['-','-','real',0, 
      df[(df.method=='real')].policy.values[0], 
      df[(df.method=='real')].cv_accuracy.values[0], '1/1'],
    ['No','No','Importance',0, 
      df[(df.method=='importance')].policy.values[0], 
      df[(df.method=='importance')].cv_accuracy.values[0], '1/1'],
    ['Yes','No','SFFS',0,
    df[(df.method=='sffs')& (df.policy_count==df[df.method=='sffs'].policy_count.max())].policy.values[0],
    df[(df.method=='sffs')& (df.policy_count==df[df.method=='sffs'].policy_count.max())].cv_accuracy.values[0],
    f'{df[df.method=="sffs"].policy_count.max()}/30'],
    ['Yes','Yes','Deep LSTM SARSA',0,
    df[(df.method=='lstm_0.0')& (df.policy_count==df[df.method=='lstm_0.0'].policy_count.max())].policy.values[0],
    df[(df.method=='lstm_0.0')& (df.policy_count==df[df.method=='lstm_0.0'].policy_count.max())].cv_accuracy.values[0],
    f'{df[df.method=="lstm_0.0"].policy_count.max()}/30'],
    ['Yes','Yes','Deep LSTM SARSA',0.5,
    df[(df.method=='lstm_0.5')& (df.policy_count==df[df.method=='lstm_0.5'].policy_count.max())].policy.values[0],
    df[(df.method=='lstm_0.5')& (df.policy_count==df[df.method=='lstm_0.5'].policy_count.max())].cv_accuracy.values[0],
    f'{df[df.method=="lstm_0.5"].policy_count.max()}/30']  ,
    ['Yes','Yes','Deep LSTM SARSA',0.67,
    df[(df.method=='lstm_0.67')& (df.policy_count==df[df.method=='lstm_0.67'].policy_count.max())].policy.values[0],
    df[(df.method=='lstm_0.67')& (df.policy_count==df[df.method=='lstm_0.67'].policy_count.max())].cv_accuracy.values[0],
    f'{df[df.method=="lstm_0.67"].policy_count.max()}/{df[df.method=="lstm_0.67"].policy_count.sum()}']
                                    ]                               ,
    columns = ['Memory','Future','Method','DF','policy','cv_accuracy','Convergence'])

In [35]:
by_Convergence

,Memory,Future,Method,DF,policy,cv_accuracy,Convergence
0,-,-,real,0.00,"[1, 2, 3, 0]",0.829,1/1
1,No,No,Importance,0.00,"[2, 3, 1, 0]",0.829,1/1
2,Yes,No,SFFS,0.00,"[2, 3, 1, 0]",0.829,30/30
3,Yes,Yes,Deep LSTM SARSA,0.00,"[2, 3, 1, 0]",0.829,14/30
4,Yes,Yes,Deep LSTM SARSA,0.50,"[2, 3, 1, 0]",0.829,12/30
5,Yes,Yes,Deep LSTM SARSA,0.67,"[2, 3, 1, 0]",0.829,10/30


In [36]:
df

,method,policy_count,policy,policy_sort,policy_sort_string,cv_accuracy
0,lstm_0.0,14,"[2, 3, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
1,lstm_0.0,8,"[3, 2, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
2,lstm_0.0,6,"[1, 3, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
3,lstm_0.0,2,"[3, 1, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
4,lstm_0.5,12,"[2, 3, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
5,lstm_0.5,10,"[3, 2, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
6,lstm_0.5,2,"[1, 3, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
7,lstm_0.5,2,"[3, 1, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
8,lstm_0.67,10,"[2, 3, 1, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829
9,lstm_0.67,4,"[1, 3, 2, 0]","[0, 1, 2, 3]","[0, 1, 2, 3]",0.829


In [37]:
df_rt

,method,avg_run_time
0,lstm_0.0,866.457377
1,lstm_0.5,4252.217634
2,lstm_0.67,5800.392548
3,sffs,18.675543


In [38]:
df.to_csv(f'Results/{str(exp)}_result2.csv')
with pd.ExcelWriter(f'Results/new5_toy_summary.xlsx', engine="openpyxl", mode="a") as writer:  

# with pd.ExcelWriter(f'Results/{dataset}_results_summary2.xlsx', engine="openpyxl", mode="a") as writer:  
    by_Convergence.to_excel(writer, sheet_name=f'{str(exp)}_by_Convergence')
    df_rt.to_excel(writer, sheet_name=f'{str(exp)}_runtime')
#     by_val_accuracy.to_excel(writer, sheet_name=f'{str(exp)}_by_val_accuracy')

In [39]:
# by_val_accuracy

In [40]:
# by_val_accuracy  = pd.DataFrame([['No','No','Importance',0, 
#   df[(df.method=='importance')].policy.values[0], 
#   df[(df.method=='importance')].test_accuracy.values[0], '1/1'],
# ['Yes','No','SFFS',0,
# df[(df.method=='sffs')& (df.val_accuracy==df[df.method=='sffs'].val_accuracy.max())].policy.values[0],
# df[(df.method=='sffs')& (df.val_accuracy==df[df.method=='sffs'].val_accuracy.max())].test_accuracy.values[0],
# f'{df[(df.method=="sffs") & (df.val_accuracy==df[df.method=="sffs"].val_accuracy.max())].policy_count.values[0]}/30'],
# # ['No','Yes','Deep SARSA',0,
# # df[(df.method=='DQN_0.0')& (df.val_accuracy==df[df.method=='DQN_0.0'].val_accuracy.max())].policy.values[0],
# # df[(df.method=='DQN_0.0')& (df.val_accuracy==df[df.method=='DQN_0.0'].val_accuracy.max())].test_accuracy.values[0],
# # f'{df[(df.method=="DQN_0.0")& (df.val_accuracy==df[df.method=="DQN_0.0"].val_accuracy.max())].policy_count.values[0]}/30'],
# # ['No','Yes','Deep SARSA',0.5,
# # df[(df.method=='DQN_0.5')& (df.val_accuracy==df[df.method=='DQN_0.5'].val_accuracy.max())].policy.values[0],
# # df[(df.method=='DQN_0.5')& (df.val_accuracy==df[df.method=='DQN_0.5'].val_accuracy.max())].test_accuracy.values[0],
# # f'{df[(df.method=="DQN_0.5")& (df.val_accuracy==df[df.method=="DQN_0.5"].val_accuracy.max())].policy_count.values[0]}/30'],
# ['Yes','Yes','Deep LSTM SARSA',0,
# df[(df.method=='lstm_0.0')& (df.val_accuracy==df[df.method=='lstm_0.0'].val_accuracy.max())].policy.values[0],
# df[(df.method=='lstm_0.0')& (df.val_accuracy==df[df.method=='lstm_0.0'].val_accuracy.max())].test_accuracy.values[0],
# f'{df[(df.method=="lstm_0.0")& (df.val_accuracy==df[df.method=="lstm_0.0"].val_accuracy.max())].policy_count.values[0]}/30'],
# ['Yes','Yes','Deep LSTM SARSA',0.5,
# df[(df.method=='lstm_0.5')& (df.val_accuracy==df[df.method=='lstm_0.5'].val_accuracy.max())].policy.values[0],
# df[(df.method=='lstm_0.5')& (df.val_accuracy==df[df.method=='lstm_0.5'].val_accuracy.max())].test_accuracy.values[0],
# f'{df[(df.method=="lstm_0.5")& (df.val_accuracy==df[df.method=="lstm_0.5"].val_accuracy.max())].policy_count.values[0]}/30'],
# ['Yes','Yes','Deep LSTM SARSA',0.67,
# df[(df.method=='lstm_0.67')& (df.val_accuracy==df[df.method=='lstm_0.67'].val_accuracy.max())].policy.values[0],
# df[(df.method=='lstm_0.67')& (df.val_accuracy==df[df.method=='lstm_0.67'].val_accuracy.max())].test_accuracy.values[0],
# f'{df[(df.method=="lstm_0.67")& (df.val_accuracy==df[df.method=="lstm_0.67"].val_accuracy.max())].policy_count.values[0]}/{df[(df.method=="lstm_0.67")].policy_count.sum()}']
# ,['-','-','real',0, 
#   df[(df.method=='real')].policy.values[0], 
#   df[(df.method=='real')].test_accuracy.values[0], '1/1']
#                                 ]                               ,
# columns = ['Memory','Future','Method','DF','policy','Test accuracy','Convergence'])